**Goal of the project**

ABC classification is a simple technique that is commonly used in inventory management and is based on the Pareto principle or 80/20 rule. This says that 80% of consequences come from 20% of causes. In ABC inventory classification, that means that 80% of product sales typically come from 20% of products, so managing the inventory on the most important 20% of products can lead to better efficiency and profits, through lower effort and fewer stockouts.

The Pareto principle applies to many things in ecommerce and marketing, including customer behaviour. As a result, ABC classification can also be used in customer segmentation, since 80% of revenue or orders often come from 20% of customers. Using ABC for customer segmentation means marketing efforts, or sales team efforts can be concentrated on the most important customers, saving labour and marketing costs by trying to treat all customers equally.

In this project, I’ll use the Pareto principle (or 80/20 rule) to group customers together in segments based on their revenue contribution.

**Load the packages**

In [52]:
# Importing libraries
import numpy as np
import pandas as pd

**Load the data**

In this project we’re going to use a [standard transactional dataset](https://www.kaggle.com/datasets/marian447/retail-store-sales-transactions) from Kaggle. This anonymized dataset includes 64.682 transactions of 5.242 SKU's sold to 22.625 customers during one year.

In [53]:
# Load dataset
df = pd.read_csv('../input/retail-store-sales-transactions/scanner_data.csv')

In [54]:
# Rename Pandas columns to lower case
df.columns = df.columns.str.lower()

In [55]:
# Examine the data
df.head()

,unnamed: 0,date,customer_id,transaction_id,sku_category,sku,quantity,sales_amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [56]:
# Overview of all variables, their datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   unnamed: 0      131706 non-null  int64  
 1   date            131706 non-null  object 
 2   customer_id     131706 non-null  int64  
 3   transaction_id  131706 non-null  int64  
 4   sku_category    131706 non-null  object 
 5   sku             131706 non-null  object 
 6   quantity        131706 non-null  float64
 7   sales_amount    131706 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 8.0+ MB


**Create a customer dataset**

We need to take the raw transaction items dataset and create a new dataset based on the customer level. This needs to include the customer ID and the total revenue. We don’t need it for the actual ABC analysis, but while we’re here we may as well calculate the total number of items and SKUs purchased by each customer. To do this we’ll aggregate the Pandas data using the agg( ) function.



In [57]:
df = df.groupby('customer_id').agg(orders = ('transaction_id', 'nunique'),
                                   skus = ('sku', 'nunique'),
                                   quantity = ('quantity', 'sum'),
                                   revenue = ('sales_amount', 'sum')).reset_index()

In [58]:
df['quantity'] = df['quantity'].apply(np.int64)

In [59]:
df = df.sort_values(by = ['revenue'], ascending = False)

In [60]:
df.head()

,customer_id,orders,skus,quantity,revenue
17470,17471,62,38,814,3985.94
17103,17104,55,86,407,3844.97
17293,17294,39,55,246,3798.39
15539,15540,38,76,165,2900.61
15676,15677,25,49,171,2765.16


**Calculate cumulative sums**

The ABC classes are calculated by calculating the cumulative sum of revenue for each customer and then calculating its the running percentage of revenue contribution. For example, the top customer here generated `$3985.94` and made up 0.25% of the total revenue. The top five customers generated 1.10% of the total revenue for the business. In the next step, we’ll classify the customers generating the top 80% of running revenue percentage to Class A.

In [61]:
df['revenue_cumsum'] = df['revenue'].cumsum()
df['revenue_total'] = df['revenue'].sum()
df['revenue_running_percentage'] = (df['revenue_cumsum'] / df['revenue_total']) * 100

In [62]:
df.head()

,customer_id,orders,skus,quantity,revenue,revenue_cumsum,revenue_total,revenue_running_percentage
17470,17471,62,38,814,3985.94,3985.94,1578038.62,0.252588
17103,17104,55,86,407,3844.97,7830.91,1578038.62,0.496243
17293,17294,39,55,246,3798.39,11629.30,1578038.62,0.736946
15539,15540,38,76,165,2900.61,14529.91,1578038.62,0.920758
15676,15677,25,49,171,2765.16,17295.07,1578038.62,1.095985


**Assign classes and ranks**

Now we have the values we need to perform the classification we can assign each customer to its ABC Class. I’m going to assign the customers generating the first 80% of the revenue to Class A. Class B will comprise the customers with 80-90%, while Class C includes the bottom 10%. Since customers can be at the top or bottom of a Class, we’ll also give each of them a rank so staff can prioritise them within classes.

In [63]:
def classify_product(percentage):
    
    if percentage > 0 and percentage <= 80:
        return 'A'
    elif percentage > 80 and percentage <= 90:
        return 'B'
    else:
        return 'C'

In [64]:
df['abc_class'] = df['revenue_running_percentage'].apply(classify_product)
df['abc_rank'] = df['revenue_running_percentage'].rank().astype(int)

In [65]:
df.head()

,customer_id,orders,skus,quantity,revenue,revenue_cumsum,revenue_total,revenue_running_percentage,abc_class,abc_rank
17470,17471,62,38,814,3985.94,3985.94,1578038.62,0.252588,A,1
17103,17104,55,86,407,3844.97,7830.91,1578038.62,0.496243,A,2
17293,17294,39,55,246,3798.39,11629.30,1578038.62,0.736946,A,3
15539,15540,38,76,165,2900.61,14529.91,1578038.62,0.920758,A,4
15676,15677,25,49,171,2765.16,17295.07,1578038.62,1.095985,A,5


**Use ABC analysis to examine the segments**

To examine the customer segments we’ll use the Pandas groupby( ) function and agg( ) again to create a summary of statistics from the dataframe. We’ll group by the abc_class and then calculate the number of unique customers, and the sum of orders, SKUs, quantity, and revenue for each ABC Class.

The data show that the 80/20 “rule” does not perfectly apply to customers in this dataset. That’s something which is pretty common. Despite the name, the Pareto rule rarely gives us an exact 80/20 split, but it’s often very close for many metrics and businesses. We find that 27% of customers generate 80% of the revenue in this business.

In [66]:
df = df.groupby('abc_class').agg(customers = ('customer_id', 'nunique'),
                                 orders = ('orders', 'sum'),
                                 skus = ('skus', 'sum'),
                                 quantity = ('quantity', 'sum'),
                                 revenue = ('revenue', 'sum')).reset_index()

In [67]:
df['retail_revenue'] = round((df['revenue'] / df['revenue'].sum()) * 100, 2)
df['retail_customers'] = round((df['customers'] / df['customers'].sum()) * 100, 2)

In [68]:
df

,abc_class,customers,orders,skus,quantity,revenue,retail_revenue,retail_customers
0,A,6172,38736,66603,141354,1262414.01,80.0,27.28
1,B,3881,9457,16420,23387,157798.67,10.0,17.15
2,C,12572,16489,23629,30780,157825.94,10.0,55.57
